In [1]:
#| default_exp core

# core

> Fill in a module description here

In [ ]:
#| export
import json, base64
from fastcore.utils import *

In [30]:
from IPython.display import Markdown
from playwrightnb import url2md,read_page
from ContextKit import read_gist

In [3]:
sak_docs = read_gist('https://gist.github.com/jph00/3d12ac8ac76c7e7b3ef62dc889284838')

In [4]:
# !pip install -q google-cloud-service-usage

In [5]:
%%ai 0 -c
I want to create a service account key file using the Python API documented in $`sak_docs`. Can you read the docs and confirm that all the needed info is there, before we begin?

1. We'll need to use the `google.cloud.iam_admin_v1` client library
2. The documentation notes that we'll need:
   - project_id: The Google Cloud project ID
   - account: The service account ID or email

In [6]:
#| export
import google.auth
from google.cloud import iam_admin_v1
from google.cloud.iam_admin_v1 import types
from google.cloud import resourcemanager_v3
from google.iam.v1 import policy_pb2

We can work with Google's IAM API by using a client, assuming you have first run `gcloud auth application-default login`.

In [8]:
cli = iam_admin_v1.IAMClient()
credentials, project_id = google.auth.default()
project = f"projects/{project_id}"
project

'projects/jph001'

We will need a "service account" with the appropriate permissions. You can check your account list like so:

In [9]:
accounts = cli.list_service_accounts(name = project)
# accounts

...and here is how to create an account:

In [10]:
account_id="aiservice"
display_name="Vertex AI Service Account"
description="Access Vertex AI"

In [11]:
svc = dict(display_name=display_name, description=description)
account = cli.create_service_account(name=project, account_id=account_id, service_account=svc)
# account

In [13]:
polcli = resourcemanager_v3.ProjectsClient()
policy = polcli.get_iam_policy(resource=project)

In [15]:
member = f"serviceAccount:{account.email}"
roles = [ "roles/aiplatform.user", "roles/servicemanagement.quotaViewer", "roles/servicemanagement.quotaAdmin" ]

In [19]:
for role in roles:
    binding = policy_pb2.Binding()
    binding.role = role
    binding.members.append(member)
    policy.bindings.append(binding)
    
polres = polcli.set_iam_policy(request={"resource": project, "policy": policy})

In [21]:
key = cli.create_service_account_key(name = f"projects/{project_id}/serviceAccounts/{account.email}")

In [34]:
keyd = json.loads(key.private_key_data.decode())
keyd['region'] = 'us-east5'
keyb = json.dumps(keyd).encode()

In [36]:
path = Path('service-account-key.json')
path.write_bytes(keyb)

2331

In [52]:
nk = json.loads(path.read_text())
# nk

In [39]:
from anthropic import AnthropicVertex, AsyncAnthropicVertex
from operator import itemgetter

In [41]:
nk['SAKF_path']=str(path)

In [44]:
gauth_proj_id,gauth_creds,region = itemgetter('project_id','SAKF_path','region')(nk)

In [45]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = gauth_creds
os.environ["GOOGLE_CLOUD_PROJECT"]           = gauth_proj_id
acli = AnthropicVertex(region=region, project_id=gauth_proj_id)

In [47]:
from claudette import *

In [50]:
cla = Client('claude-3-5-sonnet-v2@20241022', acli)

In [51]:
cla('hi')

Hello! How can I help you today?

<details>

- id: `msg_vrtx_01UfBEg5LPiemupWnBsAQh8n`
- content: `[{'text': 'Hello! How can I help you today?', 'type': 'text'}]`
- model: `claude-3-5-sonnet-v2-20241022`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'input_tokens': 8, 'output_tokens': 12}`

</details>

In [ ]:
import json, base64

# Decode the private key data
key_data = base64.b64decode(key.private_key_data)
json_key_file = json.loads(key_data)

# Save to file
with open('service-account-key.json', 'w') as f:
    json.dump(json_key_file, f)

In [84]:
# cli.delete_service_account(name=f"projects/{project_id}/serviceAccounts/aiservice@jph001.iam.gserviceaccount.com")

## Todo: quota management

In [29]:
quota_docs = read_gist('https://gist.github.com/jph00/943c51623abfe0deae65cfad2d821169')
svcuse_docs = read_gist('https://gist.github.com/jph00/042580724e98ae0cce2db50de92abd1b')

In [41]:
from google.cloud import service_usage_v1

In [64]:
%%ai 0 -c
I want to make sure we have all the quotas we need. See the docs in $`quota_docs` for examples of the quota API, and $`svcuse_docs` for service usage API docs. Do you see anything in the docs which might help here?

Yes I see some options we could use -- do you want me to outline them now?

In [75]:
scli = service_usage_v1.ServiceUsageClient()
services = scli.list_services(request={"parent": project, "filter":"state:ENABLED"})

## export -

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()